In [1]:
import pandas as pd
import numpy as np

In [2]:
countries_codes = pd.DataFrame.from_csv(r"c:\Dev\05. Jupyter\2. FDA countries clean\csv\countries_codes_curated_1Nov.csv")
countries_codes.head()

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  """Entry point for launching an IPython kernel.


,iso2,full_country_name,comment,wiki_link,population,area
0,no_code_**none**,**None**,NaN,NaN,1,1
1,af,Afghanistan,NaN,https://en.wikipedia.org/wiki/Afghanistan,1,1
2,al,Albania,NaN,https://en.wikipedia.org/wiki/Albania,1,1
3,dz,Algeria,NaN,https://en.wikipedia.org/wiki/Algeria,1,1
4,as,American Samoa,NaN,NaN,1,1


In [3]:
from bs4 import BeautifulSoup
import requests
import logging, hashlib, time, os, os.path, codecs

logging.basicConfig(filename=r'c:\Dev\04. Python\04. Sqlite converter for clinicaltrials.gov\01-Nov-2018_bis.log',level=logging.DEBUG)


def getPage(page_url, BASE_PATH, time_to_sleep_before_network = 10):
    logging.debug("Page URL:{}".format(page_url))
    print(page_url)
    
    file_to_save = BASE_PATH+  \
                        page_url.replace("/", "_").replace("?", "_").replace(":", "_")[:50]+ \
                        "_"+hashlib.sha224(page_url.encode('utf-8')).hexdigest()+ \
                        ".html"
    logging.debug("File name:{}".format(file_to_save.encode("utf-8")))
    if os.path.exists(file_to_save):
        logging.debug("File exists returning from disk")
        f=codecs.open(file_to_save, "r", encoding="utf-8")
        r = f.read()
        f.close()
        return r

    logging.debug("File does not exist: downloading")
    print("File does not exist: downloading")
    time.sleep(time_to_sleep_before_network)
    print("Sleep over")
    r = requests.get(page_url)
    #https://stackoverflow.com/questions/934160/write-to-utf-8-file-in-python
    f = codecs.open(file_to_save, "w", "utf-8")
    f.write(r.text)
    f.close()
    return r.text



In [4]:
import re


def parsePageForArea(pageLink):
    page_content = getPage(pageLink, "c:\\Dev\\05. Jupyter\\2. FDA countries clean\\", 10)
#     print(page_content)
    soup = BeautifulSoup(page_content)
    rows = soup.select(".infobox")
    
    for row in rows[0].find_all("tr"):
        found = False
#         print(row)
        area_text = "999" #This is a very abnormal return
        link = row.select("th a")
        if len(link)>0:
            if link[0].text.strip().lower() == "area":
                print(link)
                print(link[0].parent.parent.next_sibling)
                area_text = link[0].parent.parent.next_sibling.text
                found = True
        else:#for come countries: Dominican Republic, Libyan Arab Jamahiriya, Mali and United Arab Emirates
                #different setup of tags
            area_tag = row.select("th")
            if len(area_tag)>0:
                if area_tag[0].text.strip().lower() == "area":
#                     print("Area tag:{}".format(area_tag))
#                     print("Area tag:{}".format(area_tag[0].parent))
#                     print("Area tag:{}".format(area_tag[0].parent.parent))
#                     print("Area tag:{}".format(area_tag[0].parent.next_sibling))
                    area_text =  area_tag[0].parent.next_sibling.select("td")[0].text
#                     print(area_text)
                    found = True
                
        if found:
#             print("Area_text:{}".format(area_text))
            #Matching against area_text
            matchObj = re.search( r'([\d,]{1,})', area_text)#, re.M|re.I)

            if matchObj:
#                 print("matchObj.group() :{} ".format(matchObj.group()))
#                 print("matchObj.group(0) :{} ".format(matchObj.group(0))) 
#                 print("matchObj.group(1) :{} ".format(matchObj.group(1)))
                return int(matchObj.group().replace(",", ""))
    #                print("matchObj.group() :{} ".format(matchObj.group(2)))     
            else:
#                 print("No match!!")
                return -1
        
    return -2 #This is abnormal return, but it should prevent None in resulting list
    #END for row in rows[0].find_all("tr"):
                
#END def parsePageForArea(pageLink):        

In [5]:
parsePageForArea("https://en.wikipedia.org/wiki/Dominican_Republic")

https://en.wikipedia.org/wiki/Dominican_Republic

78,315 km2 (30,238 sq mi) (128th)


78315

In [6]:
result_areas_array = []
# for country in countries_codes.loc[countries_codes["wiki_link"].isnull()==False].values:
for country in countries_codes.values:
    print(country)
    if type(country[3]) is float:
        result_areas_array.append(-1)
    else:
        link = country[3]
        print("Iterating link:{}".format(link))
        areaResult = parsePageForArea(link)
        result_areas_array.append(areaResult)
        print(areaResult)

['no_code_**none**' '**None**' nan nan 1 1]
['af' 'Afghanistan' nan 'https://en.wikipedia.org/wiki/Afghanistan' 1 1]
Iterating link:https://en.wikipedia.org/wiki/Afghanistan
https://en.wikipedia.org/wiki/Afghanistan
[<a href="/wiki/Geography_of_Afghanistan" title="Geography of Afghanistan">Area </a>]
<tr class="mergedrow"><th scope="row"><div style="text-indent:-0.9em;margin-left:1.2em;font-weight:normal;">• Total</div></th><td>
652,864<sup class="reference" id="cite_ref-Area_and_administrative_Population_6-0"><a href="#cite_note-Area_and_administrative_Population-6">[5]</a></sup> km<sup>2</sup> (252,072 sq mi) (<a href="/wiki/List_of_countries_and_dependencies_by_area" title="List of countries and dependencies by area">40th</a>)</td></tr>
652864
['al' 'Albania' nan 'https://en.wikipedia.org/wiki/Albania' 1 1]
Iterating link:https://en.wikipedia.org/wiki/Albania
https://en.wikipedia.org/wiki/Albania
[<a href="/wiki/Geography_of_Albania" title="Geography of Albania">Area </a>]
<tr class

[<a href="/wiki/Geography_of_Bhutan" title="Geography of Bhutan">Area </a>]
<tr class="mergedrow"><th scope="row"><div style="text-indent:-0.9em;margin-left:1.2em;font-weight:normal;">• Total</div></th><td>
38,394 km<sup>2</sup> (14,824 sq mi)<sup class="reference" id="cite_ref-FYP9_1-0"><a href="#cite_note-FYP9-1">[1]</a></sup><sup class="reference" id="cite_ref-official_2-0"><a href="#cite_note-official-2">[2]</a></sup> (<a href="/wiki/List_of_countries_and_dependencies_by_area" title="List of countries and dependencies by area">133rd</a>)</td></tr>
38394
['bo' 'Bolivia' nan 'https://en.wikipedia.org/wiki/Bolivia' 1 1]
Iterating link:https://en.wikipedia.org/wiki/Bolivia
https://en.wikipedia.org/wiki/Bolivia
[<a href="/wiki/Geography_of_Bolivia" title="Geography of Bolivia">Area </a>]
<tr class="mergedrow"><th scope="row"><div style="text-indent:-0.9em;margin-left:1.2em;font-weight:normal;">• Total</div></th><td>
1,098,581 km<sup>2</sup> (424,164 sq mi) (<a href="/wiki/List_of_countr

[<a href="/wiki/Geography_of_Colombia" title="Geography of Colombia">Area </a>]
<tr class="mergedrow"><th scope="row"><div style="text-indent:-0.9em;margin-left:1.2em;font-weight:normal;">• Total</div></th><td>
1,141,748 km<sup>2</sup> (440,831 sq mi) (<a href="/wiki/List_of_countries_and_dependencies_by_area" title="List of countries and dependencies by area">25th</a>)</td></tr>
1141748
['km' 'Comoros' nan 'https://en.wikipedia.org/wiki/Comoros' 1 1]
Iterating link:https://en.wikipedia.org/wiki/Comoros
https://en.wikipedia.org/wiki/Comoros
[<a class="mw-redirect" href="/wiki/Geography_of_Comoros" title="Geography of Comoros">Area </a>]
<tr class="mergedrow"><th scope="row"><div style="text-indent:-0.9em;margin-left:1.2em;font-weight:normal;">• Total</div></th><td>
1,862 km<sup>2</sup> (719 sq mi) (<a href="/wiki/List_of_countries_and_dependencies_by_area" title="List of countries and dependencies by area">171th<sup>a</sup></a>)</td></tr>
1862
['cd' 'Congo' nan nan 1 1]
['cr' 'Costa Ri

[<a href="/wiki/Geography_of_Fiji" title="Geography of Fiji">Area </a>]
<tr class="mergedrow"><th scope="row"><div style="text-indent:-0.9em;margin-left:1.2em;font-weight:normal;">• Total</div></th><td>
18,274 km<sup>2</sup> (7,056 sq mi) (<a href="/wiki/List_of_countries_and_dependencies_by_area" title="List of countries and dependencies by area">151st</a>)</td></tr>
18274
['fi' 'Finland' nan 'https://en.wikipedia.org/wiki/Finland' 1 1]
Iterating link:https://en.wikipedia.org/wiki/Finland
https://en.wikipedia.org/wiki/Finland
[<a href="/wiki/Geography_of_Finland" title="Geography of Finland">Area </a>]
<tr class="mergedrow"><th scope="row"><div style="text-indent:-0.9em;margin-left:1.2em;font-weight:normal;">• Total</div></th><td>
338,424 km<sup>2</sup> (130,666 sq mi) (<a href="/wiki/List_of_countries_and_dependencies_by_area" title="List of countries and dependencies by area">64th</a>)</td></tr>
338424
['no_code_former serbia and montenegro' 'Former Serbia and Montenegro' nan
 nan 1

[<a href="/wiki/Geography_of_Honduras" title="Geography of Honduras">Area </a>]
<tr class="mergedbottomrow"><th scope="row"><div style="text-indent:-0.9em;margin-left:1.2em;font-weight:normal;">• Total</div></th><td>
112,492 km<sup>2</sup> (43,433 sq mi) (<a href="/wiki/List_of_countries_and_dependencies_by_area" title="List of countries and dependencies by area">101st</a>)</td></tr>
112492
['hk' 'Hong Kong' nan nan 1 1]
['hu' 'Hungary' nan 'https://en.wikipedia.org/wiki/Hungary' 1 1]
Iterating link:https://en.wikipedia.org/wiki/Hungary
https://en.wikipedia.org/wiki/Hungary
[<a href="/wiki/Geography_of_Hungary" title="Geography of Hungary">Area </a>]
<tr class="mergedrow"><th scope="row"><div style="text-indent:-0.9em;margin-left:1.2em;font-weight:normal;">• Total</div></th><td>
93,030<sup class="reference" id="cite_ref-area_7-0"><a href="#cite_note-area-7">[7]</a></sup> km<sup>2</sup> (35,920 sq mi) (<a href="/wiki/List_of_countries_and_dependencies_by_area" title="List of countries a

...

[<a class="mw-redirect" href="/wiki/Geography_of_Maldives" title="Geography of Maldives">Area </a>]
<tr class="mergedrow"><th scope="row"><div style="text-indent:-0.9em;margin-left:1.2em;font-weight:normal;">• Total</div></th><td>
298<sup class="reference" id="cite_ref-CIA_World_Factbook_7-0"><a href="#cite_note-CIA_World_Factbook-7">[7]</a></sup> km<sup>2</sup> (115 sq mi) (<a href="/wiki/List_of_countries_and_dependencies_by_area" title="List of countries and dependencies by area">187th</a>)</td></tr>
298
['ml' 'Mali' nan 'https://en.wikipedia.org/wiki/Mali' 1 1]
Iterating link:https://en.wikipedia.org/wiki/Mali
https://en.wikipedia.org/wiki/Mali

1,240,192 km2 (478,841 sq mi) (23rd)
1240192
['mt' 'Malta' nan 'https://en.wikipedia.org/wiki/Malta' 1 1]
Iterating link:https://en.wikipedia.org/wiki/Malta
https://en.wikipedia.org/wiki/Malta
[<a href="/wiki/Geography_of_Malta" title="Geography of Malta">Area </a>]
<tr class="mergedrow"><th scope="row"><div style="text-indent:-0.9em;margin

[<a href="/wiki/Geography_of_New_Zealand" title="Geography of New Zealand">Area </a>]
<tr class="mergedrow"><th scope="row"><div style="text-indent:-0.9em;margin-left:1.2em;font-weight:normal;">• Total</div></th><td>
268,021 km<sup>2</sup> (103,483 sq mi) (<a href="/wiki/List_of_countries_and_dependencies_by_area" title="List of countries and dependencies by area">75th</a>)</td></tr>
268021
['ni' 'Nicaragua' nan 'https://en.wikipedia.org/wiki/Nicaragua' 1 1]
Iterating link:https://en.wikipedia.org/wiki/Nicaragua
https://en.wikipedia.org/wiki/Nicaragua
[<a href="/wiki/Geography_of_Nicaragua" title="Geography of Nicaragua">Area </a>]
<tr class="mergedrow"><th scope="row"><div style="text-indent:-0.9em;margin-left:1.2em;font-weight:normal;">• Total</div></th><td>
130,375 km<sup>2</sup> (50,338 sq mi) (<a href="/wiki/List_of_countries_and_dependencies_by_area" title="List of countries and dependencies by area">96th</a>)</td></tr>
130375
['ne' 'Niger' nan 'https://en.wikipedia.org/wiki/Nige

[<a href="/wiki/Geography_of_Poland" title="Geography of Poland">Area </a>]
<tr class="mergedrow"><th scope="row"><div style="text-indent:-0.9em;margin-left:1.2em;font-weight:normal;">• Total</div></th><td>
312,696<sup class="reference" id="cite_ref-9"><a href="#cite_note-9">[7]</a></sup> km<sup>2</sup> (120,733 sq mi)<sup class="reference" id="cite_ref-11"><a href="#cite_note-11">[b]</a></sup> (<a href="/wiki/List_of_countries_and_dependencies_by_area" title="List of countries and dependencies by area">69th</a>)</td></tr>
312696
['pt' 'Portugal' nan 'https://en.wikipedia.org/wiki/Portugal' 1 1]
Iterating link:https://en.wikipedia.org/wiki/Portugal
https://en.wikipedia.org/wiki/Portugal
[<a href="/wiki/Geography_of_Portugal" title="Geography of Portugal">Area </a>]
<tr class="mergedrow"><th scope="row"><div style="text-indent:-0.9em;margin-left:1.2em;font-weight:normal;">• Total</div></th><td>
92,212 km<sup>2</sup> (35,603 sq mi)<sup class="reference" id="cite_ref-6"><a href="#cite_not

[<a href="/wiki/Geography_of_Slovenia" title="Geography of Slovenia">Area </a>]
<tr class="mergedrow"><th scope="row"><div style="text-indent:-0.9em;margin-left:1.2em;font-weight:normal;">• Total</div></th><td>
20,273 km<sup>2</sup> (7,827 sq mi) (<a href="/wiki/List_of_countries_and_dependencies_by_area" title="List of countries and dependencies by area">151st</a>)</td></tr>
20273
['sb' 'Solomon Islands' nan
 'https://en.wikipedia.org/wiki/Solomon_Islands' 1 1]
Iterating link:https://en.wikipedia.org/wiki/Solomon_Islands
https://en.wikipedia.org/wiki/Solomon_Islands
[<a class="mw-redirect" href="/wiki/Geography_of_Solomon_Islands" title="Geography of Solomon Islands">Area </a>]
<tr class="mergedrow"><th scope="row"><div style="text-indent:-0.9em;margin-left:1.2em;font-weight:normal;">• Total</div></th><td>
28,400 km<sup>2</sup> (11,000 sq mi) (<a href="/wiki/List_of_countries_and_dependencies_by_area" title="List of countries and dependencies by area">139th</a>)</td></tr>
28400
['za' 

[<a href="/wiki/Geography_of_Turkey" title="Geography of Turkey">Area </a>]
<tr class="mergedrow"><th scope="row"><div style="text-indent:-0.9em;margin-left:1.2em;font-weight:normal;">• Total</div></th><td>
783,356 km<sup>2</sup> (302,455 sq mi) (<a href="/wiki/List_of_countries_and_dependencies_by_area" title="List of countries and dependencies by area">36th</a>)</td></tr>
783356
['ug' 'Uganda' nan 'https://en.wikipedia.org/wiki/Uganda' 1 1]
Iterating link:https://en.wikipedia.org/wiki/Uganda
https://en.wikipedia.org/wiki/Uganda
[<a href="/wiki/Geography_of_Uganda" title="Geography of Uganda">Area </a>]
<tr class="mergedrow"><th scope="row"><div style="text-indent:-0.9em;margin-left:1.2em;font-weight:normal;">• Total</div></th><td>
241,038 km<sup>2</sup> (93,065 sq mi) (<a href="/wiki/List_of_countries_and_dependencies_by_area" title="List of countries and dependencies by area">79th</a>)</td></tr>
241038
['ua' 'Ukraine' nan 'https://en.wikipedia.org/wiki/Ukraine' 1 1]
Iterating link:h

In [7]:
# i=0
# while i<len(result_areas_array):
#     if result_areas_array[i] is None:
#         result_areas_array[i] = -1
#     i+=1

print(result_areas_array)

[-1, 652864, 28748, 2381741, -1, 467, 1246700, -1, 29743, -1, 7692024, 83879, 86600, -1, 765, 147570, 439, 207595, 30528, 22966, 114763, -1, 38394, 1098581, 51129, 581730, 8515767, 5765, 110993, 274200, 27834, 181035, 475442, -1, -1, 622984, 1284000, 756096, -1, 1141748, 1862, -1, 51100, 56594, 109884, 9251, -1, -1, 42933, 23200, 78315, 283561, 1010408, 21041, 28050, 45227, 1104300, -1, -1, 18274, 338424, -1, -1, 640679, -1, -1, 267667, -1, 69700, 357386, 239567, -1, 131957, -1, 348, -1, -1, 108889, -1, 36125, 214970, 27750, 0, 112492, -1, 93030, -1, 3287263, 1904569, 1648195, 437072, -1, 20770, 301340, 10991, 377973, -1, 89342, 2724900, 580367, 120540, 100363, -1, 17818, 199951, 237955, 64589, 10452, 30355, 111369, 1759541, 160, 65300, 2586, 25713, 587041, 118484, 330803, 298, 1240192, 316, -1, 1030000, 2040, -1, 1972550, 33846, 2, 1566000, 13812, -1, 710850, 801590, -1, 825615, 147181, 42508, -1, -1, 268021, 130375, 1267000, 923768, -1, 385203, 309500, 881913, 459, 6020, -1, 75417, 4

In [8]:
def parsePageForPopulation(pageLink):
    page_content = getPage(pageLink, "c:\\Dev\\05. Jupyter\\2. FDA countries clean\\", 10)
#     print(page_content)
    soup = BeautifulSoup(page_content)
    rows = soup.select(".infobox")
    
    for row in rows[0].find_all("tr"):
        found = False
#         print(row)
        area_text = "999" #This is a very abnormal return
        link = row.select("th a")
        if len(link)>0:
            if link[0].text.strip().lower() == "population":
                print(link)
                print(link[0].parent.parent.next_sibling)
                area_text = link[0].parent.parent.next_sibling.select("td")[0].text
                found = True
#         else:#for come countries: Dominican Republic, Libyan Arab Jamahiriya, Mali and United Arab Emirates
#                 #different setup of tags
#             area_tag = row.select("th")
#             if len(area_tag)>0:
#                 if area_tag[0].text.strip().lower() == "population":
# #                     print("Area tag:{}".format(area_tag))
# #                     print("Area tag:{}".format(area_tag[0].parent))
# #                     print("Area tag:{}".format(area_tag[0].parent.parent))
# #                     print("Area tag:{}".format(area_tag[0].parent.next_sibling))
#                     area_text =  area_tag[0].parent.next_sibling.select("td")[0].text
#                     print(area_text)
#                     found = True
                
        if found:
            print("Area_text:{}".format(area_text))
            #Matching against area_text
            matchObj = re.search( r'([\d,]{1,})', area_text)#, re.M|re.I)

            if matchObj:
#                 print("matchObj.group() :{} ".format(matchObj.group()))
#                 print("matchObj.group(0) :{} ".format(matchObj.group(0))) 
#                 print("matchObj.group(1) :{} ".format(matchObj.group(1)))
                return int(matchObj.group().replace(",", ""))
    #                print("matchObj.group() :{} ".format(matchObj.group(2)))     
            else:
#                 print("No match!!")
                return -1
        
    return -2 #This is abnormal return, but it should prevent None in resulting list
    #END for row in rows[0].find_all("tr"):
# parsePageForPopulation("https://en.wikipedia.org/wiki/Afghanistan")

In [10]:
result_population_array = []
# for country in countries_codes.loc[countries_codes["wiki_link"].isnull()==False].values:
for country in countries_codes.values:
    print(country)
    if type(country[3]) is float:
        result_population_array.append(-1)
    else:
        link = country[3]
        print("Iterating link:{}".format(link))
        areaResult = parsePageForPopulation(link)
        result_population_array.append(areaResult)
        print(areaResult)

['no_code_**none**' '**None**' nan nan 1 1]
['af' 'Afghanistan' nan 'https://en.wikipedia.org/wiki/Afghanistan' 1 1]
Iterating link:https://en.wikipedia.org/wiki/Afghanistan
https://en.wikipedia.org/wiki/Afghanistan
[<a href="/wiki/Demographics_of_Afghanistan" title="Demographics of Afghanistan">Population</a>]
<tr class="mergedrow"><th scope="row"><div style="text-indent:-0.9em;margin-left:1.2em;font-weight:normal;">• 2016 estimate</div></th><td>
34,656,032<sup class="reference" id="cite_ref-UN_WPP_7-0"><a href="#cite_note-UN_WPP-7">[6]</a></sup> (<a href="/wiki/List_of_countries_and_dependencies_by_population" title="List of countries and dependencies by population">40th</a>)</td></tr>
Area_text:
34,656,032[6] (40th)
34656032
['al' 'Albania' nan 'https://en.wikipedia.org/wiki/Albania' 1 1]
Iterating link:https://en.wikipedia.org/wiki/Albania
https://en.wikipedia.org/wiki/Albania
[<a href="/wiki/Demographics_of_Albania" title="Demographics of Albania">Population</a>]
<tr class="merged

[<a href="/wiki/Demographics_of_Belarus" title="Demographics of Belarus">Population</a>]
<tr class="mergedrow"><th scope="row"><div style="text-indent:-0.9em;margin-left:1.2em;font-weight:normal;">• 2018 estimate</div></th><td>
9,491,800 <img alt="Decrease" data-file-height="300" data-file-width="300" height="11" src="//upload.wikimedia.org/wikipedia/commons/thumb/e/ed/Decrease2.svg/11px-Decrease2.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/e/ed/Decrease2.svg/17px-Decrease2.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/e/ed/Decrease2.svg/22px-Decrease2.svg.png 2x" title="Decrease" width="11"/><sup class="reference" id="cite_ref-ReferenceA_2-0"><a href="#cite_note-ReferenceA-2">[2]</a></sup> (<a href="/wiki/List_of_countries_and_dependencies_by_population" title="List of countries and dependencies by population">93rd</a>)</td></tr>
Area_text:
9,491,800 [2] (93rd)
9491800
['be' 'Belgium' nan 'https://en.wikipedia.org/wiki/Belgium' 1 1]
Iterating link:ht

[<a href="/wiki/Demographics_of_Burundi" title="Demographics of Burundi">Population</a>]
<tr class="mergedrow"><th scope="row"><div style="text-indent:-0.9em;margin-left:1.2em;font-weight:normal;">• 2016 estimate</div></th><td>
10,524,117<sup class="reference" id="cite_ref-UN_WPP_9-0"><a href="#cite_note-UN_WPP-9">[9]</a></sup> (<a href="/wiki/List_of_countries_and_dependencies_by_population" title="List of countries and dependencies by population">86th</a>)</td></tr>
Area_text:
10,524,117[9] (86th)
10524117
['kh' 'Cambodia' nan 'https://en.wikipedia.org/wiki/Cambodia' 1 1]
Iterating link:https://en.wikipedia.org/wiki/Cambodia
https://en.wikipedia.org/wiki/Cambodia
[<a href="/wiki/Demographics_of_Cambodia" title="Demographics of Cambodia">Population</a>]
<tr class="mergedrow"><th scope="row"><div style="text-indent:-0.9em;margin-left:1.2em;font-weight:normal;">• 2018 estimate</div></th><td>
16,245,729</td></tr>
Area_text:
16,245,729
16245729
['cm' 'Cameroon' nan 'https://en.wikipedia.o

[<a href="/wiki/Demographics_of_Denmark" title="Demographics of Denmark">Population</a>]
<tr class="mergedrow"><th scope="row"><div style="text-indent:-0.9em;margin-left:1.2em;font-weight:normal;">• 2018 estimate</div></th><td>
<img alt="Increase" data-file-height="300" data-file-width="300" height="11" src="//upload.wikimedia.org/wikipedia/commons/thumb/b/b0/Increase2.svg/11px-Increase2.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/b/b0/Increase2.svg/17px-Increase2.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/b/b0/Increase2.svg/22px-Increase2.svg.png 2x" title="Increase" width="11"/> 5,789,957<sup class="reference" id="cite_ref-8"><a href="#cite_note-8">[4]</a></sup> (<a class="mw-redirect" href="/wiki/List_of_countries_by_population" title="List of countries by population">112th</a>)</td></tr>
Area_text:
 5,789,957[4] (112th)
5789957
['dj' 'Djibouti' nan 'https://en.wikipedia.org/wiki/Djibouti' 1 1]
Iterating link:https://en.wikipedia.org/wiki/Djibou

[<a href="/wiki/Demographics_of_France" title="Demographics of France">Population</a>]
<tr class="mergedrow"><th scope="row"><div style="text-indent:-0.9em;margin-left:1.2em;font-weight:normal;">• October 2018 estimate</div></th><td>
<img alt="Increase" data-file-height="300" data-file-width="300" height="11" src="//upload.wikimedia.org/wikipedia/commons/thumb/b/b0/Increase2.svg/11px-Increase2.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/b/b0/Increase2.svg/17px-Increase2.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/b/b0/Increase2.svg/22px-Increase2.svg.png 2x" title="Increase" width="11"/>67,348,000<sup class="reference" id="cite_ref-11"><a href="#cite_note-11">[5]</a></sup> (<a href="/wiki/List_of_countries_and_dependencies_by_population" title="List of countries and dependencies by population">21st</a>)</td></tr>
Area_text:
67,348,000[5] (21st)
67348000
['gf' 'French Guiana' nan nan 1 1]
['pf' 'French Polynesia' nan nan 1 1]
['ga' 'Gabon' nan 'https

...

[<a href="/wiki/Demographics_of_Oman" title="Demographics of Oman">Population</a>]
<tr class="mergedrow"><th scope="row"><div style="text-indent:-0.9em;margin-left:1.2em;font-weight:normal;">• 2016 estimate</div></th><td>
4,424,762<sup class="reference" id="cite_ref-UN_WPP_3-0"><a href="#cite_note-UN_WPP-3">[3]</a></sup> (<a href="/wiki/List_of_countries_and_dependencies_by_population" title="List of countries and dependencies by population">125th</a>)</td></tr>
Area_text:
4,424,762[3] (125th)
4424762
['pk' 'Pakistan' nan 'https://en.wikipedia.org/wiki/Pakistan' 1 1]
Iterating link:https://en.wikipedia.org/wiki/Pakistan
https://en.wikipedia.org/wiki/Pakistan
[<a href="/wiki/Demographics_of_Pakistan" title="Demographics of Pakistan">Population</a>]
<tr class="mergedrow"><th scope="row"><div style="text-indent:-0.9em;margin-left:1.2em;font-weight:normal;">• 2017 census</div></th><td>
212,742,631 <sup class="reference" id="cite_ref-census_20-0"><a href="#cite_note-census-20">[19]</a></sup

[<a href="/wiki/Demographics_of_Qatar" title="Demographics of Qatar">Population</a>]
<tr class="mergedrow"><th scope="row"><div style="text-indent:-0.9em;margin-left:1.2em;font-weight:normal;">• 2017 estimate</div></th><td>
2,641,669<sup class="reference" id="cite_ref-qsa-Aug13_2-0"><a href="#cite_note-qsa-Aug13-2">[2]</a></sup> (<a href="/wiki/List_of_countries_and_dependencies_by_population" title="List of countries and dependencies by population">140th</a>)</td></tr>
Area_text:
2,641,669[2] (140th)
2641669
['ro' 'Romania' nan 'https://en.wikipedia.org/wiki/Romania' 1 1]
Iterating link:https://en.wikipedia.org/wiki/Romania
https://en.wikipedia.org/wiki/Romania
[<a href="/wiki/Demographics_of_Romania" title="Demographics of Romania">Population</a>]
<tr class="mergedrow"><th scope="row"><div style="text-indent:-0.9em;margin-left:1.2em;font-weight:normal;">• 2017 estimate</div></th><td>
19,638,000 <img alt="Decrease" data-file-height="300" data-file-width="300" height="11" src="//upload

[<a href="/wiki/Demographics_of_Singapore" title="Demographics of Singapore">Population</a>]
<tr class="mergedrow"><th scope="row"><div style="text-indent:-0.9em;margin-left:1.2em;font-weight:normal;">• 2017 estimate</div></th><td>
5,612,300<sup class="reference" id="cite_ref-Population_and_Population_Structure_2017_5-0"><a href="#cite_note-Population_and_Population_Structure_2017-5">[5]</a></sup> (<a href="/wiki/List_of_countries_and_dependencies_by_population" title="List of countries and dependencies by population">113th</a>)</td></tr>
Area_text:
5,612,300[5] (113th)
5612300
['sk' 'Slovakia' nan 'https://en.wikipedia.org/wiki/Slovakia' 1 1]
Iterating link:https://en.wikipedia.org/wiki/Slovakia
https://en.wikipedia.org/wiki/Slovakia
[<a href="/wiki/Demographics_of_Slovakia" title="Demographics of Slovakia">Population</a>]
<tr class="mergedrow"><th scope="row"><div style="text-indent:-0.9em;margin-left:1.2em;font-weight:normal;">• 2018 estimate</div></th><td>
5,445,087 <img alt="Incre

[<a href="/wiki/Demographics_of_Syria" title="Demographics of Syria">Population</a>]
<tr class="mergedrow"><th scope="row"><div style="text-indent:-0.9em;margin-left:1.2em;font-weight:normal;">• 2018 estimate</div></th><td>
<img alt="Increase" data-file-height="300" data-file-width="300" height="11" src="//upload.wikimedia.org/wikipedia/commons/thumb/b/b0/Increase2.svg/11px-Increase2.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/b/b0/Increase2.svg/17px-Increase2.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/b/b0/Increase2.svg/22px-Increase2.svg.png 2x" title="Increase" width="11"/> 18,284,407 (<a href="/wiki/List_of_countries_and_dependencies_by_population" title="List of countries and dependencies by population">63rd</a>)</td></tr>
Area_text:
 18,284,407 (63rd)
18284407
['tw' 'Taiwan' nan nan 1 1]
['tj' 'Tajikistan' nan 'https://en.wikipedia.org/wiki/Tajikistan' 1 1]
Iterating link:https://en.wikipedia.org/wiki/Tajikistan
https://en.wikipedia.org/wiki/

[<a class="mw-redirect" href="/wiki/Demographics_of_the_United_Kingdom" title="Demographics of the United Kingdom">Population</a>]
<tr class="mergedrow"><th scope="row"><div style="text-indent:-0.9em;margin-left:1.2em;font-weight:normal;">• 2017 estimate</div></th><td>
<img alt="Increase" data-file-height="300" data-file-width="300" height="11" src="//upload.wikimedia.org/wikipedia/commons/thumb/b/b0/Increase2.svg/11px-Increase2.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/b/b0/Increase2.svg/17px-Increase2.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/b/b0/Increase2.svg/22px-Increase2.svg.png 2x" title="Increase" width="11"/> 66,040,229<sup class="reference" id="cite_ref-13"><a href="#cite_note-13">[9]</a></sup> (<a href="/wiki/List_of_countries_and_dependencies_by_population" title="List of countries and dependencies by population">22nd</a>)</td></tr>
Area_text:
 66,040,229[9] (22nd)
66040229
['us' 'United States of America' nan
 'https://en.wikipedia

In [11]:
print(result_population_array)

[-1, 34656032, 2876591, 42200000, -1, 77281, 25789024, -1, 2924816, -1, 25110100, 8823054, 9898100, -1, 1425171, 162951560, 277821, 9491800, 11420163, 387879, 10872298, -1, 797765, 11248864, 3511372, 2250260, 210147125, 417200, 7050034, 20107509, 10524117, 16245729, 23439189, -1, -1, 4594621, 13670084, 17574003, -1, 49996445, 795601, -1, 4857274, 4154200, 11221060, 1170125, -1, -1, 5789957, 942333, 10735896, 16385068, 94798827, 6344722, 1221490, 1319133, 102403196, -1, -1, 912241, 5520535, -1, -1, 67348000, -1, -1, 1979786, -1, 3718200, 82800000, 28308301, -1, 10768477, -1, 107317, -1, -1, 17263239, -1, 1815698, 773303, 10847334, 1000, 9112867, -1, 9797561, -1, 1324171354, 261115456, 81672300, 37202572, -1, 8938750, 60483973, 2890299, 126440000, -1, 10171480, 18311700, 49125325, 25368620, 51446201, -1, 4052584, 6019480, 6758353, 1925800, 6006668, 2203821, 4503000, -2, 38111, 2800667, 602005, 2103721, 24894551, 18091575, 32049700, 427756, -2, 475700, -1, 4301018, 1262132, -1, 123675325,

In [12]:
def parsePageForFlagLink(pageLink):
    page_content = getPage(pageLink, "c:\\Dev\\05. Jupyter\\2. FDA countries clean\\", 10)
#     print(page_content)
    soup = BeautifulSoup(page_content)
    rows = soup.select(".infobox")
    
    flag_text_a = rows[0].select("tr > td > div > div > div > a")
#     print(flag_text_a)
    flag_link = "999"
    for flag in flag_text_a:
#         print("")
#         print("")
#         print("")
#         print(flag)
        found = False
#         if len(flag_text_a) > 0:
        if flag.text.lower() == "flag":
            flag_link = flag.parent.parent.select("div > a > img")[0].attrs["src"]
#             print(flag_link)
#             print(flag.parent.parent.previous_sibling.find("a > img"))
#             flag_link = flag.parent.parent.previous_sibling.find("a > img")
#             print(flag_link)
            found = True
    

        
    return flag_link #This is abnormal return, but it should prevent None in resulting list
    
# parsePageForFlagLink("https://en.wikipedia.org/wiki/Afghanistan")

In [13]:
result_flag_array = []
# for country in countries_codes.loc[countries_codes["wiki_link"].isnull()==False].values:
for country in countries_codes.values:
#     print(country)
    if type(country[3]) is float:
        result_flag_array.append(-1)
    else:
        link = country[3]
        print("Iterating link:{}".format(link))
        flag_link = parsePageForFlagLink(link)
        result_flag_array.append(flag_link)
        print(flag_link)

['no_code_**none**' '**None**' nan nan 1 1]
['af' 'Afghanistan' nan 'https://en.wikipedia.org/wiki/Afghanistan' 1 1]
Iterating link:https://en.wikipedia.org/wiki/Afghanistan
https://en.wikipedia.org/wiki/Afghanistan
//upload.wikimedia.org/wikipedia/commons/thumb/9/9a/Flag_of_Afghanistan.svg/125px-Flag_of_Afghanistan.svg.png
//upload.wikimedia.org/wikipedia/commons/thumb/9/9a/Flag_of_Afghanistan.svg/125px-Flag_of_Afghanistan.svg.png
['al' 'Albania' nan 'https://en.wikipedia.org/wiki/Albania' 1 1]
Iterating link:https://en.wikipedia.org/wiki/Albania
https://en.wikipedia.org/wiki/Albania
//upload.wikimedia.org/wikipedia/commons/thumb/3/36/Flag_of_Albania.svg/125px-Flag_of_Albania.svg.png
//upload.wikimedia.org/wikipedia/commons/thumb/3/36/Flag_of_Albania.svg/125px-Flag_of_Albania.svg.png
['dz' 'Algeria' nan 'https://en.wikipedia.org/wiki/Algeria' 1 1]
Iterating link:https://en.wikipedia.org/wiki/Algeria
https://en.wikipedia.org/wiki/Algeria
//upload.wikimedia.org/wikipedia/commons/thumb/7

//upload.wikimedia.org/wikipedia/commons/thumb/9/9c/Flag_of_Brunei.svg/125px-Flag_of_Brunei.svg.png
//upload.wikimedia.org/wikipedia/commons/thumb/9/9c/Flag_of_Brunei.svg/125px-Flag_of_Brunei.svg.png
['bg' 'Bulgaria' nan 'https://en.wikipedia.org/wiki/Bulgaria' 1 1]
Iterating link:https://en.wikipedia.org/wiki/Bulgaria
https://en.wikipedia.org/wiki/Bulgaria
//upload.wikimedia.org/wikipedia/commons/thumb/9/9a/Flag_of_Bulgaria.svg/125px-Flag_of_Bulgaria.svg.png
//upload.wikimedia.org/wikipedia/commons/thumb/9/9a/Flag_of_Bulgaria.svg/125px-Flag_of_Bulgaria.svg.png
['bf' 'Burkina Faso' nan 'https://en.wikipedia.org/wiki/Burkina_Faso' 1 1]
Iterating link:https://en.wikipedia.org/wiki/Burkina_Faso
https://en.wikipedia.org/wiki/Burkina_Faso
//upload.wikimedia.org/wikipedia/commons/thumb/3/31/Flag_of_Burkina_Faso.svg/125px-Flag_of_Burkina_Faso.svg.png
//upload.wikimedia.org/wikipedia/commons/thumb/3/31/Flag_of_Burkina_Faso.svg/125px-Flag_of_Burkina_Faso.svg.png
['bi' 'Burundi' nan 'https://en.

//upload.wikimedia.org/wikipedia/commons/thumb/8/8f/Flag_of_Estonia.svg/125px-Flag_of_Estonia.svg.png
//upload.wikimedia.org/wikipedia/commons/thumb/8/8f/Flag_of_Estonia.svg/125px-Flag_of_Estonia.svg.png
['et' 'Ethiopia' nan 'https://en.wikipedia.org/wiki/Ethiopia' 1 1]
Iterating link:https://en.wikipedia.org/wiki/Ethiopia
https://en.wikipedia.org/wiki/Ethiopia
//upload.wikimedia.org/wikipedia/commons/thumb/7/71/Flag_of_Ethiopia.svg/125px-Flag_of_Ethiopia.svg.png
//upload.wikimedia.org/wikipedia/commons/thumb/7/71/Flag_of_Ethiopia.svg/125px-Flag_of_Ethiopia.svg.png
['fo' 'Faroe Islands' nan nan 1 1]
['fm' 'Federated States of Micronesia' nan nan 1 1]
['fj' 'Fiji' nan 'https://en.wikipedia.org/wiki/Fiji' 1 1]
Iterating link:https://en.wikipedia.org/wiki/Fiji
https://en.wikipedia.org/wiki/Fiji
//upload.wikimedia.org/wikipedia/commons/thumb/b/ba/Flag_of_Fiji.svg/125px-Flag_of_Fiji.svg.png
//upload.wikimedia.org/wikipedia/commons/thumb/b/ba/Flag_of_Fiji.svg/125px-Flag_of_Fiji.svg.png
['fi'

//upload.wikimedia.org/wikipedia/commons/thumb/d/d4/Flag_of_Israel.svg/125px-Flag_of_Israel.svg.png
//upload.wikimedia.org/wikipedia/commons/thumb/d/d4/Flag_of_Israel.svg/125px-Flag_of_Israel.svg.png
['it' 'Italy' nan 'https://en.wikipedia.org/wiki/Italy' 1 1]
Iterating link:https://en.wikipedia.org/wiki/Italy
https://en.wikipedia.org/wiki/Italy
//upload.wikimedia.org/wikipedia/en/thumb/0/03/Flag_of_Italy.svg/125px-Flag_of_Italy.svg.png
//upload.wikimedia.org/wikipedia/en/thumb/0/03/Flag_of_Italy.svg/125px-Flag_of_Italy.svg.png
['jm' 'Jamaica' nan 'https://en.wikipedia.org/wiki/Jamaica' 1 1]
Iterating link:https://en.wikipedia.org/wiki/Jamaica
https://en.wikipedia.org/wiki/Jamaica
//upload.wikimedia.org/wikipedia/commons/thumb/0/0a/Flag_of_Jamaica.svg/125px-Flag_of_Jamaica.svg.png
//upload.wikimedia.org/wikipedia/commons/thumb/0/0a/Flag_of_Jamaica.svg/125px-Flag_of_Jamaica.svg.png
['jp' 'Japan' nan 'https://en.wikipedia.org/wiki/Japan' 1 1]
Iterating link:https://en.wikipedia.org/wiki/

//upload.wikimedia.org/wikipedia/commons/thumb/6/66/Flag_of_Malaysia.svg/125px-Flag_of_Malaysia.svg.png
//upload.wikimedia.org/wikipedia/commons/thumb/6/66/Flag_of_Malaysia.svg/125px-Flag_of_Malaysia.svg.png
['mv' 'Maldives' nan 'https://en.wikipedia.org/wiki/Maldives' 1 1]
Iterating link:https://en.wikipedia.org/wiki/Maldives
https://en.wikipedia.org/wiki/Maldives
//upload.wikimedia.org/wikipedia/commons/thumb/0/0f/Flag_of_Maldives.svg/125px-Flag_of_Maldives.svg.png
//upload.wikimedia.org/wikipedia/commons/thumb/0/0f/Flag_of_Maldives.svg/125px-Flag_of_Maldives.svg.png
['ml' 'Mali' nan 'https://en.wikipedia.org/wiki/Mali' 1 1]
Iterating link:https://en.wikipedia.org/wiki/Mali
https://en.wikipedia.org/wiki/Mali
//upload.wikimedia.org/wikipedia/commons/thumb/9/92/Flag_of_Mali.svg/125px-Flag_of_Mali.svg.png
//upload.wikimedia.org/wikipedia/commons/thumb/9/92/Flag_of_Mali.svg/125px-Flag_of_Mali.svg.png
['mt' 'Malta' nan 'https://en.wikipedia.org/wiki/Malta' 1 1]
Iterating link:https://en.w

//upload.wikimedia.org/wikipedia/commons/thumb/3/32/Flag_of_Pakistan.svg/125px-Flag_of_Pakistan.svg.png
//upload.wikimedia.org/wikipedia/commons/thumb/3/32/Flag_of_Pakistan.svg/125px-Flag_of_Pakistan.svg.png
['pw' 'Palau' nan 'https://en.wikipedia.org/wiki/Palau' 1 1]
Iterating link:https://en.wikipedia.org/wiki/Palau
https://en.wikipedia.org/wiki/Palau
//upload.wikimedia.org/wikipedia/commons/thumb/4/48/Flag_of_Palau.svg/125px-Flag_of_Palau.svg.png
//upload.wikimedia.org/wikipedia/commons/thumb/4/48/Flag_of_Palau.svg/125px-Flag_of_Palau.svg.png
['no_code_palestinian territories, occupied'
 'Palestinian Territories, Occupied' nan
 'https://en.wikipedia.org/wiki/State_of_Palestine' 1 1]
Iterating link:https://en.wikipedia.org/wiki/State_of_Palestine
https://en.wikipedia.org/wiki/State_of_Palestine
//upload.wikimedia.org/wikipedia/commons/thumb/0/00/Flag_of_Palestine.svg/125px-Flag_of_Palestine.svg.png
//upload.wikimedia.org/wikipedia/commons/thumb/0/00/Flag_of_Palestine.svg/125px-Flag_o

//upload.wikimedia.org/wikipedia/commons/thumb/f/f0/Flag_of_Slovenia.svg/125px-Flag_of_Slovenia.svg.png
//upload.wikimedia.org/wikipedia/commons/thumb/f/f0/Flag_of_Slovenia.svg/125px-Flag_of_Slovenia.svg.png
['sb' 'Solomon Islands' nan
 'https://en.wikipedia.org/wiki/Solomon_Islands' 1 1]
Iterating link:https://en.wikipedia.org/wiki/Solomon_Islands
https://en.wikipedia.org/wiki/Solomon_Islands
//upload.wikimedia.org/wikipedia/commons/thumb/7/74/Flag_of_the_Solomon_Islands.svg/125px-Flag_of_the_Solomon_Islands.svg.png
//upload.wikimedia.org/wikipedia/commons/thumb/7/74/Flag_of_the_Solomon_Islands.svg/125px-Flag_of_the_Solomon_Islands.svg.png
['za' 'South Africa' nan 'https://en.wikipedia.org/wiki/South_Africa' 1 1]
Iterating link:https://en.wikipedia.org/wiki/South_Africa
https://en.wikipedia.org/wiki/South_Africa
999
['es' 'Spain' nan 'https://en.wikipedia.org/wiki/Spain' 1 1]
Iterating link:https://en.wikipedia.org/wiki/Spain
https://en.wikipedia.org/wiki/Spain
//upload.wikimedia.org/

//upload.wikimedia.org/wikipedia/commons/thumb/6/6e/Flag_of_Vanuatu_%28official%29.svg/125px-Flag_of_Vanuatu_%28official%29.svg.png
//upload.wikimedia.org/wikipedia/commons/thumb/6/6e/Flag_of_Vanuatu_%28official%29.svg/125px-Flag_of_Vanuatu_%28official%29.svg.png
['ve' 'Venezuela' nan nan 1 1]
['vn' 'Vietnam' nan 'https://en.wikipedia.org/wiki/Venezuela' 1 1]
Iterating link:https://en.wikipedia.org/wiki/Venezuela
https://en.wikipedia.org/wiki/Venezuela
//upload.wikimedia.org/wikipedia/commons/thumb/7/7b/Flag_of_Venezuela_%28state%29.svg/125px-Flag_of_Venezuela_%28state%29.svg.png
//upload.wikimedia.org/wikipedia/commons/thumb/7/7b/Flag_of_Venezuela_%28state%29.svg/125px-Flag_of_Venezuela_%28state%29.svg.png
['vi' 'Virgin Islands (U.S.)' nan 'https://en.wikipedia.org/wiki/Vietnam'
 1 1]
Iterating link:https://en.wikipedia.org/wiki/Vietnam
https://en.wikipedia.org/wiki/Vietnam
//upload.wikimedia.org/wikipedia/commons/thumb/2/21/Flag_of_Vietnam.svg/125px-Flag_of_Vietnam.svg.png
//upload.w

In [14]:
print(result_flag_array)

[-1, '//upload.wikimedia.org/wikipedia/commons/thumb/9/9a/Flag_of_Afghanistan.svg/125px-Flag_of_Afghanistan.svg.png', '//upload.wikimedia.org/wikipedia/commons/thumb/3/36/Flag_of_Albania.svg/125px-Flag_of_Albania.svg.png', '//upload.wikimedia.org/wikipedia/commons/thumb/7/77/Flag_of_Algeria.svg/125px-Flag_of_Algeria.svg.png', -1, '//upload.wikimedia.org/wikipedia/commons/thumb/1/19/Flag_of_Andorra.svg/125px-Flag_of_Andorra.svg.png', '//upload.wikimedia.org/wikipedia/commons/thumb/9/9d/Flag_of_Angola.svg/125px-Flag_of_Angola.svg.png', -1, '//upload.wikimedia.org/wikipedia/commons/thumb/2/2f/Flag_of_Armenia.svg/125px-Flag_of_Armenia.svg.png', -1, '//upload.wikimedia.org/wikipedia/commons/thumb/8/88/Flag_of_Australia_%28converted%29.svg/125px-Flag_of_Australia_%28converted%29.svg.png', '//upload.wikimedia.org/wikipedia/commons/thumb/4/41/Flag_of_Austria.svg/125px-Flag_of_Austria.svg.png', '//upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Flag_of_Azerbaijan.svg/125px-Flag_of_Azerbaijan.

In [15]:
countries_codes["area"]=np.array(result_areas_array).astype(int)
countries_codes["population"]=np.array(result_population_array).astype(int)
countries_codes["flag"] = result_flag_array
countries_codes.head()
# countries_codes.loc[countries_codes.full_country_name == "Qatar"].head()
# countries_codes.loc[countries_codes.area.isnull()].head()

,iso2,full_country_name,comment,wiki_link,population,area,flag
0,no_code_**none**,**None**,NaN,NaN,-1,-1,-1
1,af,Afghanistan,NaN,https://en.wikipedia.org/wiki/Afghanistan,34656032,652864,//upload.wikimedia.org/wikipedia/commons/thumb...
2,al,Albania,NaN,https://en.wikipedia.org/wiki/Albania,2876591,28748,//upload.wikimedia.org/wikipedia/commons/thumb...
3,dz,Algeria,NaN,https://en.wikipedia.org/wiki/Algeria,42200000,2381741,//upload.wikimedia.org/wikipedia/commons/thumb...
4,as,American Samoa,NaN,NaN,-1,-1,-1


In [20]:
from IPython.display import Audio
sound_file = r"c:\Windows\media\Alarm01.wav"
Audio(sound_file, autoplay=True)

In [21]:
countries_codes.to_csv(r"c:\Dev\05. Jupyter\2. FDA countries clean\csv\countries_codes_area_1Nov.csv")